## Text Classification Prediction Explanations

In this notebook, we'll see some methods of explaining the predictions of a text classification model.

In [1]:
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, roc_auc_score, precision_recall_fscore_support, precision_recall_curve

from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter 
from scipy.spatial.distance import pdist


In [2]:
import accuracy

In [3]:
from sklearn import metrics

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return metrics.accuracy_score(y_true=p.label_ids, y_pred=preds)


In [4]:
import torch
import os
from datasets import Dataset
from accelerate import Accelerator
import tqdm as notebook_tqdm

In [5]:
from bs4 import BeautifulSoup
import re
from sklearn import preprocessing

In [6]:
#  from gensim.models import word2vec

In [7]:
from transformers import AutoConfig, Pipeline, RobertaTokenizer, RobertaModel, AutoTokenizer, DistilBertModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, DistilBertConfig, DistilBertTokenizer, DistilBertTokenizerFast, DistilBertPreTrainedModel, DistilBertForTokenClassification, DistilBertForSequenceClassification
import evaluate
from torch.utils.data import DataLoader

In [8]:
from sklearn import metrics

In [9]:
import shap

In [10]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

model_path = "./GatorTronMedium_Last20Layers_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()
# model.to("cuda")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

MegatronBertForSequenceClassification(
  (bert): MegatronBertModel(
    (embeddings): MegatronBertEmbeddings(
      (word_embeddings): Embedding(50176, 2560, padding_idx=0)
      (position_embeddings): Embedding(512, 2560)
      (token_type_embeddings): Embedding(2, 2560)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MegatronBertEncoder(
      (layer): ModuleList(
        (0-47): 48 x MegatronBertLayer(
          (attention): MegatronBertAttention(
            (ln): LayerNorm((2560,), eps=1e-12, elementwise_affine=True)
            (self): MegatronBertSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MegatronBertSelfOutput(
              (dense): Linear(in_features=2560, out_featur

In [11]:
val = pd.read_csv("../data/HospHospReadmissionWithin30DayssWithin30DaysOfDischarge_Xtest_Ypred_df_HospitalDischargeNotesCSV_exported_GatorTron_Last20Layers1iter.csv")
val = val.rename(columns={'HospReadmissionWithin30DayssWithin30Days': 'HospReadmissionWithin30DaysWithin30Days'})
val

,text,HospReadmissionWithin30DaysWithin30Days
0,name unit no admission date discharge date dat...,1
1,name unit no admission date discharge date dat...,0
2,name unit no admission date discharge date dat...,0
3,name unit no admission date discharge date dat...,0
4,name unit no admission date discharge date dat...,0
...,...,...
37167,name unit no admission date discharge date dat...,0
37168,name unit no admission date discharge date dat...,0
37169,name unit no admission date discharge date dat...,0
37170,name unit no admission date discharge date dat...,0


In [12]:
val = val.sample(3, random_state=224)
val

,text,HospReadmissionWithin30DaysWithin30Days
9259,name unit no admission date discharge date dat...,1
28132,name unit no admission date discharge date dat...,1
3779,name unit no admission date discharge date dat...,0


In [13]:
def shap_predict(texts):
    if isinstance(texts, np.ndarray):
        is_pre_tokenized = isinstance(texts[0], (list, np.ndarray))
        texts = texts.tolist()
    else:
        is_pre_tokenized = isinstance(texts[0], list)

    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128,
        is_split_into_words=is_pre_tokenized
    )

    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        output = model(**inputs).logits

    return torch.nn.functional.softmax(output, dim=-1).cpu().numpy()



masker = shap.maskers.Text(tokenizer)
explainer = shap.Explainer(shap_predict, masker)

In [14]:
class_names = [0, 1]  # options/labels for classification

In [15]:
import emoji
import pandas as pd

def clean_text(text):
    no_emoji = emoji.replace_emoji(text, replace='')
    cleaned = no_emoji.encode("utf-8", "ignore").decode("utf-8", "ignore")
    return cleaned

all_shap = []
print("Running SHAP on validation samples...")

for idx, row in val.iterrows():
    text = clean_text(str(row["text"]))
    true_label = row["HospReadmissionWithin30DaysWithin30Days"]
    print(f"Explaining discharge notes {idx+1}: {text[:50]}...")

    shap_values = explainer([text])
    pred_label = class_names[np.argmax(shap_values.values[0].sum(axis=0))]

    for word, value in zip(shap_values.data[0], shap_values.values[0][np.argmax(shap_values.values[0].sum(axis=0))]):
        all_shap.append({
            "text": text,
            "true_label": true_label,
            "pred_label": pred_label,
            "word": word,
            "shap_value": value
        })

df_shap = pd.DataFrame(all_shap)
print("SHAP explanations complete.")

Running SHAP on validation samples...
Explaining discharge notes 9260: name unit no admission date discharge date date of...


  0%|          | 0/498 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df_shap_clean = df_shap[df_shap["word"].str.len() > 3]
df_shap_clean = df_shap_clean[df_shap_clean["word"].str[0].str.isalpha()]

top_words = df_shap_clean.groupby("word")["shap_value"].mean().sort_values(ascending=False).head(20)

plt.figure(figsize=(7,4))
sns.barplot(y=top_words.index, x=top_words.values)
plt.title("Top 20 Words by Average SHAP Value")
plt.xlabel("Average SHAP Value")
plt.ylabel("Word")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from wordcloud import WordCloud

word_freq = df_shap.groupby("word")["shap_value"].mean().to_dict()
wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(word_freq)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("SHAP Word Importance Cloud")
plt.show()

In [ ]:
from nltk.corpus import wordnet
import nltk
import random
nltk.download("wordnet")

def synonym_replace(text):
    words = text.split()
    new_words = []
    for word in words:
        syns = wordnet.synsets(word)
        if syns and random.random() < 0.2:
            lemmas = syns[0].lemma_names()
            if lemmas:
                new_words.append(lemmas[0].replace("_", " "))
                continue
        new_words.append(word)
    return " ".join(new_words)

stability_scores = []

for i in range(len(val)):
    text = val.iloc[i]["tweet"]
    perturbed = synonym_replace(text)

    shap_orig = explainer([text])
    shap_pert = explainer([perturbed])

    idx = np.argmax(shap_orig.values[0].sum(axis=0))
    words_orig = set(shap_orig.data[0])
    words_pert = set(shap_pert.data[0])
    jaccard = len(words_orig & words_pert) / len(words_orig | words_pert)
    stability_scores.append(jaccard)

print(f"Average Jaccard similarity over {len(val)} perturbed explanations: {np.mean(stability_scores):.3f}")

In [ ]:
stop here

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

In [ ]:
HospitalDischargeNotes = pd.read_csv("../data/Merged_DischargeNotes_FollowedByHospReadmissionWithin30DaysOfDischarge_sparse.csv")

In [ ]:
HospitalDischargeNotes  

,text,HospReadmissionWithin30Days
0,\nName: ___ Unit No: _...,0.0
1,\nName: ___ Unit No: _...,1.0
2,\nName: ___ Unit No: _...,1.0
3,\nName: ___ Unit No: ___\n...,0.0
4,\nName: ___. Unit No: ___\n \nAdm...,1.0
...,...,...
185855,\nName: ___ Unit No: __...,1.0
185856,\nName: ___ Unit No: __...,0.0
185857,\nName: ___ Unit No: __...,0.0
185858,\nName: ___ Unit No: ___...,0.0


In [ ]:
class TextCleaner():
    def __init__(self):
        pass
    
    def clean_text(self, text):
        text = text.lower() # lowercase everything
        text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
        text = re.sub(r'https*\S+', ' ', text) # remove links
        text = re.sub(r'http*\S+', ' ', text)
        text = re.sub(r'<.*?_:>', '', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
        text = re.sub(r"\s+", " ", text).strip()
        # cleaning up text
        text = re.sub(r'\'\w+', '', text) 
        text = re.sub(r'\w*\d+\w*', '', text)
        text = re.sub(r'\s{2,}', ' ', text)
        text = re.sub(r'\s[^\w\s]\s', '', text)
        return text
    
cleaner = TextCleaner()
HospitalDischargeNotes['cleaned_text'] = HospitalDischargeNotes['text'].apply(cleaner.clean_text)

In [ ]:
le = preprocessing.LabelEncoder()
HospitalDischargeNotes['labels'] = le.fit_transform(HospitalDischargeNotes['HospReadmissionWithin30Days'].tolist())

In [ ]:
HospitalDischargeNotes = HospitalDischargeNotes.drop(['text', 'HospReadmissionWithin30Days'], axis=1)

In [ ]:
train_df, test_df = train_test_split(HospitalDischargeNotes, test_size=0.2, stratify=HospitalDischargeNotes['labels'], random_state=321, shuffle=True)

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("UFNLP/gatortron-medium")
config=AutoConfig.from_pretrained('UFNLP/gatortron-medium')

# Convert datasets to tokenized format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def tokenize_data(examples):
    return tokenizer(examples["cleaned_text"], truncation=True, max_length=512, return_tensors="pt", padding=True)

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/148688 [00:00<?, ? examples/s]

Map:   0%|          | 0/37172 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'
PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0

In [ ]:
# Load pre-trained DistilBERT model (or another model) for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("UFNLP/gatortron-medium", num_labels=len(le.classes_))  

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at UFNLP/gatortron-medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Freeze the base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
# Unfreeze the last 20 layers of the transformer
for i in [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]:   # for i in [antepenultimate_layer_index, penultimate_layer_index, last_layer_index] if you want to unfreeze specific layers:
    for param in model.base_model.encoder.layer[i].parameters():
        param.requires_grad = True

In [ ]:

# Prepare data collator for padding sequences
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
# DataLoader with dynamic padding
train_dataloader = DataLoader(
    tokenized_train,
    shuffle=True,
    batch_size=8,  # adjust based on your GPU
    collate_fn=data_collator
)

val_dataloader = DataLoader(
    tokenized_test,
    batch_size=8, 
    collate_fn=data_collator
)

In [ ]:


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    warmup_steps=500,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
    # gradient_accumulation_steps=2,  # Useful for limited VRAM
    # push_to_hub=True
)

# Define Trainer object for training the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    processing_class=tokenizer,
    #tokenizer=tokenizer,
    data_collator=data_collator
  # compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the trained model
trainer.save_model('model')

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


trainer.predict(tokenized_test)

In [ ]:
y_pred = np.argmax(trainer.predict(tokenized_test).predictions, axis=1)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
y_true = tokenized_test['labels']

In [ ]:
print(classification_report(y_true, y_pred, zero_division = 0, target_names=['No Recorded Hospital Readmission Within 30 Days', 'Readmitted to Hospital Within 30 Days']))

                                                 precision    recall  f1-score   support

No Recorded Hospital Readmission Within 30 Days       0.71      0.90      0.80     24575
          Readmitted to Hospital Within 30 Days       0.60      0.29      0.39     12597

                                       accuracy                           0.69     37172
                                      macro avg       0.66      0.60      0.59     37172
                                   weighted avg       0.67      0.69      0.66     37172



print(classification_report(
    y_true=tokenized_test['labels'],
    y_pred=np.argmax(trainer.predict(tokenized_test).predictions, axis=1),
    target_names=le.classes_
))

print(classification_report(
    y_true=tokenized_test['labels'],
    y_pred,
    target_names=le.classes_
))

In [ ]:
tokenized_train

Dataset({
    features: ['cleaned_text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 148688
})

In [ ]:
test_df = test_df.rename(columns={'cleaned_text': 'text', 'labels': 'HospReadmissionWithin30DaysWithin30Days'})
test_df

,text,HospReadmissionWithin30DayssWithin30Days
16313,name unit no admission date discharge date dat...,1
63720,name unit no admission date discharge date dat...,0
155633,name unit no admission date discharge date dat...,0
132922,name unit no admission date discharge date dat...,0
27775,name unit no admission date discharge date dat...,0
...,...,...
15792,name unit no admission date discharge date dat...,0
19798,name unit no admission date discharge date dat...,0
138452,name unit no admission date discharge date dat...,0
46128,name unit no admission date discharge date dat...,0


In [ ]:
test_df.to_csv('../data/HospHospReadmissionWithin30DayssWithin30DaysOfDischarge_Xtest_Ypred_df_HospitalDischargeNotesCSV_exported_GatorTron_Last20Layers1iter.csv', index=False)